# Import the libraries

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

# Get URL and send GET request

In [19]:
url='https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels?page=1'
response=requests.get(url)
if response.status_code==200:
    print("Request Successful")
else:
    print("Request Failed")

Request Successful


# Parse the HTML Content

In [24]:
#create a soup object to parse the html content
soup=BeautifulSoup(response.content,'html.parser')
print(soup)

<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head>
<title>All Time Favorite Romance Novels (5374 books)</title>
<meta content="5,353 books based on 12615 votes: Pride and Prejudice by Jane Austen, Fifty Shades of Grey by E.L. James, Beautiful Disaster by Jamie McGuire, Twilight b..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels" rel="canonical"/>
<script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();
 </script>
<script type="text/javascript">
    var ue_mid = "A1PQBFHBHS6YH1";
    var ue_sn = "www.goodreads.com";
    var ue_furl = "fls-na.amazon.com";
    var ue_sid = "123-4964859-6084493";
    var ue_id = "ADKN9NZ7WMH2T0D42T0W";

    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:argumen

# Extract Book Details for Page 1

In [25]:
# Find all book containers on the page
book_containers=soup.find_all('tr',itemtype="http://schema.org/Book")

# Create empty lists to hold book information
names=[]
authors=[]
ratings=[]
avgscores=[]
scores=[]
votes=[]

# Iterate through each book container
for container in book_containers:
     # Extract book title
    name = container.find('a',class_="bookTitle").text.strip()
    names.append(name)
    
    # Extract author name
    author = container.find('a',class_="authorName").text.strip()
    authors.append(author)
    
    # Extract rating and average score
    scoring = container.find('span',class_="greyText smallText uitext").text.strip().split()
    ratings.append(scoring[4])
    avgscores.append(scoring[0])
    
    # Extract number of scores and votes
    voting= container.find('span',class_="smallText uitext").text.strip().split()  
    scores.append(voting[1])
    votes.append(voting[3])

# Create a DataFrame to store the extracted data
df=pd.DataFrame({'Book Title': names,
                 'Author': authors,
                'Ratings': ratings,
                'Average Score': avgscores,
                'Score': scores,
                'Votes':votes})

df.head()

,Book Title,Author,Ratings,Average Score,Score,Votes
0,Pride and Prejudice,Jane Austen,"4,254,342",4.29,"242,754,","2,456"
1,"Fifty Shades of Grey (Fifty Shades, #1)",E.L. James,"2,598,074",3.66,"231,163,","2,345"
2,"Beautiful Disaster (Beautiful, #1)",Jamie McGuire,"681,175",4.01,"216,238,","2,194"
3,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,"6,554,968",3.65,"140,016,","1,437"
4,"The Notebook (The Notebook, #1)",Nicholas Sparks,"1,686,046",4.15,"99,788,","1,025"


# Extract Book Details for All Pages

In [27]:
# Create empty lists to hold book information
names=[]
authors=[]
ratings=[]
avgscores=[]
scores=[]
votes=[]

# Loop through each page of the Goodreads list
for page_num in range(1,55):
    url=f'https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels?page={page_num}'
    response=requests.get(url)
    
    # Parse the HTML content of the page
    soup=BeautifulSoup(response.content,'html.parser')
    
    # Find all book containers on the page
    book_containers = soup.find_all('tr', itemtype="http://schema.org/Book")
    
    # Iterate through each book container
    for container in book_containers:
         # Extract book title
        name = container.find('a',class_="bookTitle").text.strip()
        names.append(name)
    
        # Extract author name
        author = container.find('a',class_="authorName").text.strip()
        authors.append(author)
    
        # Extract rating and average score
        scoring = container.find('span',class_="greyText smallText uitext").text.strip().split()
        ratings.append(scoring[4])
        avgscores.append(scoring[0])
    
        # Extract number of scores and votes
        voting= container.find('span',class_="smallText uitext").text.strip().split()  
        scores.append(voting[1])
        votes.append(voting[3])
        
# Print message indicating completion of data extraction
print("Data extraction completed.")

Data extraction completed.


# Export the data

In [30]:
# Create a dictionary to hold the data
# Create a DataFrame to store the extracted data
df=pd.DataFrame({'Book Title': names,
                 'Author': authors,
                'Ratings': ratings,
                'Average Score': avgscores,
                'Score': scores,
                'Votes':votes})

df.head(10)

,Book Title,Author,Ratings,Average Score,Score,Votes
0,Pride and Prejudice,Jane Austen,"4,254,354",4.29,"242,754,","2,456"
1,"Fifty Shades of Grey (Fifty Shades, #1)",E.L. James,"2,598,081",3.66,"231,163,","2,345"
2,"Beautiful Disaster (Beautiful, #1)",Jamie McGuire,"681,175",4.01,"216,238,","2,194"
3,"Twilight (The Twilight Saga, #1)",Stephenie Meyer,"6,554,989",3.65,"140,016,","1,437"
4,"The Notebook (The Notebook, #1)",Nicholas Sparks,"1,686,046",4.15,"99,788,","1,025"
5,"Perfect Chemistry (Perfect Chemistry, #1)",Simone Elkeles,"259,624",4.04,"98,391,","1,005"
6,"Outlander (Outlander, #1)",Diana Gabaldon,"1,049,387",4.26,"92,123,",939
7,Jane Eyre,Charlotte Brontë,"2,089,498",4.15,"88,963,",913
8,"Thoughtless (Thoughtless, #1)",S.C. Stephens,"140,151",4.08,"82,516,",849
9,"Bared to You (Crossfire, #1)",Sylvia Day,"525,820",4.14,"79,994,",832


In [31]:
# Save the DataFrame to a CSV file
df.to_csv("GoodRead_Book.csv", index=False)
print("DataFrame saved to GoodRead_Book.csv")


DataFrame saved to GoodRead_Book.csv
